# KR-WordRank
- 비지도학습으로 한국어 텍스트에서 키워드를 자동으로 추출해주는 라이브러리
- **주요 가정 : 단어 주변에는 단어가 등장하며, 올바른 단어는 주위의 많은 단어들과 연결되어 있다. 따라서 단어는 주위 단어들에 의하여 단어 점수가 보강(reinforced)된다.**
- [KR-WordRank Reference](https://github.com/lovit/KR-WordRank)
- [Setup]
```
pip install krwordrank
```

### 전체 파일 합치기

In [1]:
import os
os.chdir('../data/')

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm

year = [str(x) for x in range(2011, 2022)]   # 2011 ~ 2021
df = pd.DataFrame(columns = ['year', 'fileName', 'text', 'preprocessing', 'keywords'])
df

,year,fileName,text,preprocessing,keywords


In [4]:
for y in tqdm(year):
    fileList = os.listdir('./' + y)
    for file in fileList:
        with open('./' + y + '/' + file, 'r') as f:
            textsList = f.read().splitlines()
        text = ' '.join(textsList)
        df_slice = pd.Series([y, file, text, 0, 0], index = df.columns)
        df = df.append(df_slice, ignore_index = True)
df.to_csv('all_before_preprocessing.csv', index = False)

100%|██████████| 11/11 [00:05<00:00,  2.09it/s]


In [5]:
df.head()

,year,fileName,text,preprocessing,keywords
0,2011,HPC_kor_2010.pdf.txt,"H HPC, Our life, PC Su Sta in a bility R Ou...",0,0
1,2011,2010_STX팬오션_지속가능보고서.pdf.txt,2010 STX 팬오션 지속가능경영보고서 NOW and FROM NOW About...,0,0
2,2011,풀무원 2010 지속경영보고서.pdf.txt,(cid:0) (cid:0) 주 식 회 사 풀 무 원 홀 딩 스 서 울 ...,0,0
3,2011,웅진그룹_2011환경사회보고서.pdf.txt,웅 진 ● 2 0 1 www.woongjin.com 1 환 ...,0,0
4,2011,DGB금융그룹_SR_2010_2011.pdf.txt,DGB금융그룹 2010/2011 지속가능경영보고서 TOWARD THE WORLD ...,0,0


In [6]:
len(df)

1043

### 정규표현식으로 전처리

In [7]:
def preprocessing_regex(text):
    import re
    
    afterText = re.sub('[^가-힣a-zA-Z]', ' ', text)
    afterText = re.sub('(\(cid:[0-9]+\))', ' ', afterText)
    afterText = re.sub('/\r\n|\r|\n/', '', afterText)       # 개행문자 제거
    afterText = re.sub(' +', ' ', afterText)                # 연속된 공백 한개로
    
    return afterText

In [8]:
df['preprocessing'] = df['text'].apply(preprocessing_regex)
df.head()

,year,fileName,text,preprocessing,keywords
0,2011,HPC_kor_2010.pdf.txt,"H HPC, Our life, PC Su Sta in a bility R Ou...",H HPC Our life PC Su Sta in a bility R Our fut...,0
1,2011,2010_STX팬오션_지속가능보고서.pdf.txt,2010 STX 팬오션 지속가능경영보고서 NOW and FROM NOW About...,STX 팬오션 지속가능경영보고서 NOW and FROM NOW About This...,0
2,2011,풀무원 2010 지속경영보고서.pdf.txt,(cid:0) (cid:0) 주 식 회 사 풀 무 원 홀 딩 스 서 울 ...,cid cid 주 식 회 사 풀 무 원 홀 딩 스 서 울 시 강 남 구 수 서 동...,0
3,2011,웅진그룹_2011환경사회보고서.pdf.txt,웅 진 ● 2 0 1 www.woongjin.com 1 환 ...,웅 진 www woongjin com 환 경 보 고 서 웅 진 환 경 보 고 서 웅...,0
4,2011,DGB금융그룹_SR_2010_2011.pdf.txt,DGB금융그룹 2010/2011 지속가능경영보고서 TOWARD THE WORLD ...,DGB금융그룹 지속가능경영보고서 TOWARD THE WORLD Better Toge...,0


In [26]:
df.to_csv('all_before_extracting_keywords.csv', index = False)

### 키워드 추출

In [9]:
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize      # 숫자, 영어 포함 여부
from krwordrank.word import summarize_with_keywords
import numpy as np

def extract_keywords(text):
    error_cnt = 0
    try:
        min_count = 3       # 단어의 최소 출현 빈도수 (그래프 생성 시) [5, 3]
        max_length = 30     # 단어의 최대 길이 [10, 30]
        verbose = False     # [True, False]
        wordrank_extractor = KRWordRank(min_count, max_length, verbose)

        beta = 0.85         # PageRank의 decaying factor beta
        max_iter = 30       # [10, 30]

        text = normalize(text, english = True, number = False).split(' ')
        keywords, rank, graph = wordrank_extractor.extract(text, beta, max_iter)

    #     stopwords = {'대한','의견','단어'}
    #     keywords = summarize_with_keywords(texts, min_count = 3, max_length = 30, beta = 0.85, max_iter = 30, stopwords = stopwords, verbose = True)
        keywords = summarize_with_keywords(text)    # with default arguments
        result = sorted(keywords.items(), key = lambda x:x[1], reverse = True)

        return [result[x][0] for x in range(len(result))]
    
    except:
        error_cnt += 1
        print(error_cnt)
        
        return np.nan

In [10]:
df['keywords'] = df['preprocessing'].apply(extract_keywords)
df.head()

1


,year,fileName,text,preprocessing,keywords
0,2011,HPC_kor_2010.pdf.txt,"H HPC, Our life, PC Su Sta in a bility R Ou...",H HPC Our life PC Su Sta in a bility R Our fut...,"[있습니, 지속, 활동, 환경, 호남석, 제품, 녹색, 공정, 보고, 에너, 협력,..."
1,2011,2010_STX팬오션_지속가능보고서.pdf.txt,2010 STX 팬오션 지속가능경영보고서 NOW and FROM NOW About...,STX 팬오션 지속가능경영보고서 NOW and FROM NOW About This...,"[있습니, STX팬, 지속, 환경, 임직, 선박, 배출, 활동, 다양, 확보, 노력..."
2,2011,풀무원 2010 지속경영보고서.pdf.txt,(cid:0) (cid:0) 주 식 회 사 풀 무 원 홀 딩 스 서 울 ...,cid cid 주 식 회 사 풀 무 원 홀 딩 스 서 울 시 강 남 구 수 서 동...,"[풀무원, 제품, 사회, 지속경, 고객, 환경, 지표, 바른, 브랜드, 관리, GR..."
3,2011,웅진그룹_2011환경사회보고서.pdf.txt,웅 진 ● 2 0 1 www.woongjin.com 1 환 ...,웅 진 www woongjin com 환 경 보 고 서 웅 진 환 경 보 고 서 웅...,"[웅진, 환경, 있습니, 에너, 계열, 사용, 제품, 지속, 사업, 온실가, 배출,..."
4,2011,DGB금융그룹_SR_2010_2011.pdf.txt,DGB금융그룹 2010/2011 지속가능경영보고서 TOWARD THE WORLD ...,DGB금융그룹 지속가능경영보고서 TOWARD THE WORLD Better Toge...,"[지역, 지속, DGB, 있습, 사회, 대구, 금융, 고객, 보고, 환경, 지원, ..."


In [15]:
df.text[0][:500]

'H HPC, Our life,  PC Su Sta in a bility R   Our future ePo R t 20 10 HPC Sustainability Report 2010 다. 니 습 었 되 쇄 인 로  으 름 기 에 콩 081 이 8 종 5- 경  54 환 2) 한 친 S  (0 용 U 사 PL 고지를  ESIGN 는 재생 d by D 고서 gne 본 보 Desi 서울시 동작구 신대방동 395-67 롯데타워빌딩 스마트폰 QR코드 어플리케이션을 다운받아 스캔하시면  모바일 설문조사에 참여하실 수 있습니다. 보고서 개요 본 보고서는 호남석유화학의 지속가능한 발전을 위한 노력과 결실을 담은 네 번째 보고서입니 보고 기간 및 범위 다. 2010 지속가능경영보고서는 이해관계자에게 친근한 기업으로 다가서고자 이해관계자 의 호남석유화학은 2007년부터 매년 지속가능경영보고서를 발 간하고 있으며, 본 보고서는 2010년 1월 1일부터 12월 31일 견을 적극 수렴하였으며, 호남석유화학 제'

In [13]:
df.preprocessing[0][:500]

'H HPC Our life PC Su Sta in a bility R Our future ePo R t HPC Sustainability Report 다 니 습 었 되 쇄 인 로 으 름 기 에 콩 이 종 경 환 한 친 S 용 U 사 PL 고지를 ESIGN 는 재생 d by D 고서 gne 본 보 Desi 서울시 동작구 신대방동 롯데타워빌딩 스마트폰 QR코드 어플리케이션을 다운받아 스캔하시면 모바일 설문조사에 참여하실 수 있습니다 보고서 개요 본 보고서는 호남석유화학의 지속가능한 발전을 위한 노력과 결실을 담은 네 번째 보고서입니 보고 기간 및 범위 다 지속가능경영보고서는 이해관계자에게 친근한 기업으로 다가서고자 이해관계자 의 호남석유화학은 년부터 매년 지속가능경영보고서를 발 간하고 있으며 본 보고서는 년 월 일부터 월 일 견을 적극 수렴하였으며 호남석유화학 제품으로 만들어지는 좀 더 편리하고 즐거운 일상의 모 까지의 활동을 주로 담고 있습니다 이해관계자 여러분의 이해 습을 담았습니다'

### 키워드 추출 오류값 1개 삭제

In [4]:
df = pd.read_csv('all_after_keywords.csv')
df

,year,fileName,text,preprocessing,keywords
0,2011,HPC_kor_2010.pdf.txt,"H HPC, Our life, PC Su Sta in a bility R Ou...",H HPC Our life PC Su Sta in a bility R Our fut...,"['있습니', '지속', '활동', '환경', '호남석', '제품', '녹색', '..."
1,2011,2010_STX팬오션_지속가능보고서.pdf.txt,2010 STX 팬오션 지속가능경영보고서 NOW and FROM NOW About...,STX 팬오션 지속가능경영보고서 NOW and FROM NOW About This...,"['있습니', 'STX팬', '지속', '환경', '임직', '선박', '배출', ..."
2,2011,풀무원 2010 지속경영보고서.pdf.txt,(cid:0) (cid:0) 주 식 회 사 풀 무 원 홀 딩 스 서 울 ...,cid cid 주 식 회 사 풀 무 원 홀 딩 스 서 울 시 강 남 구 수 서 동...,"['풀무원', '제품', '사회', '지속경', '고객', '환경', '지표', '..."
3,2011,웅진그룹_2011환경사회보고서.pdf.txt,웅 진 ● 2 0 1 www.woongjin.com 1 환 ...,웅 진 www woongjin com 환 경 보 고 서 웅 진 환 경 보 고 서 웅...,"['웅진', '환경', '있습니', '에너', '계열', '사용', '제품', '지..."
4,2011,DGB금융그룹_SR_2010_2011.pdf.txt,DGB금융그룹 2010/2011 지속가능경영보고서 TOWARD THE WORLD ...,DGB금융그룹 지속가능경영보고서 TOWARD THE WORLD Better Toge...,"['지역', '지속', 'DGB', '있습', '사회', '대구', '금융', '고..."
...,...,...,...,...,...
1038,2021,국립생태원.pdf.txt,None ABOUT THIS REPORT 보고서 개요 CONTENTS 국립생태원은...,None ABOUT THIS REPORT 보고서 개요 CONTENTS 국립생태원은 ...,"['생태', '국립생태원', '있습니', '환경', '안전', '사회', '자연',..."
1039,2021,LX하우시스.pdf.txt,"LX하우시스 지속가능경영보고서 2021 Nature, Human and Space ...",LX하우시스 지속가능경영보고서 Nature Human and Space LX Hau...,"['환경', '있습니', '안전', '지속', '협력', '사업', '고객', '제..."
1040,2021,두산인프라코어.pdf.txt,DOOSAN INFRACORE 2020 Integrated Report file ...,DOOSAN INFRACORE Integrated Report file Doosan...,"['있습', '협력', '두산', '지속', '사업', '진행', '기술', '기반..."
1041,2021,SK가스.pdf.txt,None Cover Story 경남 창녕 소재 우포늪은 람사르 협약 지정 습지이자 ...,None Cover Story 경남 창녕 소재 우포늪은 람사르 협약 지정 습지이자 ...,"['사업', '지속', '가스', '있습니', '사회', '리스', '배출', '이..."


In [5]:
df[df.keywords.isna()]

,year,fileName,text,preprocessing,keywords
553,2016,CJ제일제당.pdf.txt,NaN,NaN,NaN


In [6]:
df.drop(553, inplace = True)
df.reset_index(drop = True, inplace = True)
len(df)

1042

In [7]:
df.to_csv('all_after_keywords.csv', index = False)